In [205]:
# from pathlib import Path
from sklearn.model_selection import train_test_split
from pprint import pprint
import pandas as pd
import numpy as np
import sys
import os
import gc
import re
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import RMSprop

In [216]:
batch_size = 128
epochs = 5

In [207]:
max_num_features = 10
# max_data_size = 
# pad_size = 1
# boundary_letter = -1
space_letter = 0
space_padded_tokens = []

In [208]:
# training_data = pd.read_csv("en_train.csv", nrows=1000)
training_data = pd.read_csv("en_train.csv")
max_data_size = len(training_data)

In [209]:
encoded_classes = pd.factorize(training_data['class'])
# print_data = 100000
x_data = []
labels = encoded_classes[1]
y_data = encoded_classes[0]
gc.collect()
count = 0

In [210]:
for before_value in training_data['before'].values:
    row = np.ones(max_num_features, dtype=int) * space_letter
    for before_value_char, i in zip(list(str(before_value)), np.arange(max_num_features)):
        row[i] = ord(before_value_char)
    count+=1
    x_data.append(row)

In [211]:
def make_flat_context_windows(data, pad_size, max_num_features, boundary_letter):
    pad = np.zeros(shape=max_num_features)
    #create array of pad arrays
    pads = [pad for _ in  np.arange(pad_size)]
    data = pads + data + pads
    flattened_context_windows = []
    for lower_bound in np.arange(len(data) - pad_size * 2):
        flattened_context_window = []
        
        context_window_size = pad_size * 2 + 1
        upper_bound = lower_bound + context_window_size
        context_window = data[lower_bound:upper_bound]
        
        for word in context_window:
            flattened_context_window.append([boundary_letter])
            flattened_context_window.append(word)
        flattened_context_window.append([boundary_letter])
        
        #append window list to list of window lists
        flattened_context_windows.append([int(word) for row in flattened_context_window for word in row])
    return flattened_context_windows

In [212]:
# x_data = np.array(context_window_transform(x_data, pad_size))
x_data = x_data[:max_data_size]
y_data = y_data[:max_data_size]
x_data = np.array(make_flat_context_windows(x_data, pad_size = 1, max_num_features= max_num_features, boundary_letter=-1))
gc.collect()


0

In [213]:
x_train = np.array(x_data)
y_train = np.array(y_data)
gc.collect()

0

In [214]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1, random_state=2017)

In [215]:
num_classes = len(labels)

In [217]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

In [218]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_valid = np.reshape(x_valid, (x_valid.shape[0], x_valid.shape[1], 1))

In [219]:
model = Sequential()
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, input_shape=((max_num_features * 3) + 4, 1)))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(labels), activation='softmax'))

In [220]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 128)               66560     
_________________________________________________________________
dense_15 (Dense)             (None, 32)                4128      
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 16)                528       
Total params: 71,216
Trainable params: 71,216
Non-trainable params: 0
_________________________________________________________________


In [221]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [222]:
# print(x_train.shape, y_train.shape)

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_valid, y_valid))

Train on 8926596 samples, validate on 991845 samples
Epoch 1/5
7905920/8926596 [=========================>....] - ETA: 2056s - loss: 0.1268 - acc: 0.9715